In [208]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle

In [255]:
plate = 13
date_init = datetime(2020,7,1,19,57)
dates_datetime = [date_init+timedelta(hours=4)*i for i in range(24)]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime]

In [12]:
dates[0]

'0701_1957'

In [252]:
class Experiment():
    def __init__(self,plate):
        self.plate = plate
    def load(self,dates,local=False,pickle=False):
        self.dates=dates
        if local:
            paths=[f'Data/graph_{date}_{plate}_full_labeled.csv' for date in dates]
            paths_connection = [f'Data/graph_{date}_{plate}_connection.csv' for date in dates[:-1]]
        else:
            paths = [get_path(date,self.plate,True,extension='_full_labeled.csv') for date in dates]
            paths_connection = [get_path(date,plate,True,extension='_connection.csv') for date in dates[:-1]]
        nx_graph_poss=[]
        for path in paths:
            nx_graph_poss.append(generate_nx_graph(pd.read_csv(path,
                                converters={'origin_pos' : transform_list,'end_pos' : transform_list,
                                            'pixel_list' : ast.literal_eval}),labeled=True))
        from_tip_growth_pattern=[]
        for path in paths_connection:
            from_tip_growth_pattern.append(from_connection_tab(pd.read_csv(path)))
        nx_graphs=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
        poss = [nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
        nx_graph_clean=[]
        for graph in nx_graphs:
            S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
            len_connected=[len(nx_graph.nodes) for nx_graph in S]
            nx_graph_clean.append(S[np.argmax(len_connected)])
        skeletons=[]
        for nx_graph in nx_graph_clean:
            skeletons.append(generate_skeleton(nx_graph,dim=(20800, 46000)))
        connections = [c[0] for c in from_tip_growth_pattern]
        growth_patterns = [c[1] for c in from_tip_growth_pattern]
        self.positions=poss
        self.nx_graph=nx_graph_clean
        self.connections = connections
        self.growth_patterns = growth_patterns
        self.skeletons=skeletons
        labels = {node for g in self.nx_graph for node in g}
        self.nodes=[]
        for label in labels:
            self.nodes.append(Node(label,self))
        
    def copy(self,experiment):
        self.positions=experiment.positions
        self.nx_graph=experiment.nx_graph
        self.connections = experiment.connections
        self.growth_patterns = experiment.growth_patterns
        self.skeletons=experiment.skeletons
    def save(self,path=f'Data/'):
        print('wow')
        from_tip_growth_pattern_tab=[]
        for date in self.dates[:-1]:
            from_tip_growth_pattern_tab.append(pd.read_csv(get_path(date,plate,True,extension='_connection.csv')))
        for i, date in enumerate(self.dates[:-1]):
            from_tip_growth_pattern_tab[i].to_csv(path+f'connection_{date}_{plate}.csv')
        tabs_labeled=[]
        for date in self.dates:
            tabs_labeled.append(pd.read_csv(get_path(date,plate,True,extension='_full_labeled.csv'),
                                        converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))
        for i, date in enumerate(self.dates):
            tabs_labeled[i].to_csv(path+f'graph_{date}_{plate}_full_labeled.csv')
            sio.savemat(path+f'graph_{date}_{plate}_full_labeled.mat', {name: col.values for name, col in tabs_labeled[i].items()})
    def pickle_save(self):
        pickle.dump(self,open( f'Data/'+f"experiment_{plate}.pick", "wb" ))
    def pickle_load(self):
        self = pickle.load( open( f'Data/'+f"experiment_{plate}.pick", "rb" ) )
    def get_node(self,label):
        return(Node(label,self))
    def get_edge(self,begin,end):
        return(Edge(begin,end,self))
    def compress_skeleton(self,t,factor):
        shape=self.skeletons[t].shape
        final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
        for pixel in self.skeletons[t].keys():
            x=min(round(pixel[0]/factor),shape[0]//factor-1)
            y=min(round(pixel[1]/factor),shape[1]//factor-1)
            final_picture[x,y]+=1
        return(final_picture>=1)
    def plot(self,ts,labels=[],relabel_t=lambda x:x,relabel_tp1=lambda x:x, shift=(0,0),compress=1,save='',time=None,power=4):
        right = 0.90
        top = 0.90
        if len(save)>=1:
            fig=plt.figure(figsize=(14,12))
            size = 10
        else:
            fig = plt.figure()
            size = 5
        ax = fig.add_subplot(111)
        np.random.seed(0)
        scale=1/len(ts)
        final_image=scale*experiment.compress_skeleton(ts[0],compress)
        for i,t in enumerate(ts[1:]):
            final_image=np.power(final_image**power+((i+2)*scale*experiment.compress_skeleton(t,compress))**power,1/power)
        ax.imshow(final_image, cmap='plasma',interpolation='none',vmin=0,vmax=1)
#         bbox_props1 = dict(boxstyle="circle", fc="grey")
#         bbox_props2 = dict(boxstyle="circle", fc="white")
#         ax.text(right, top, time,
#             horizontalalignment='right',
#             verticalalignment='bottom',
#             transform=ax.transAxes,color='white')
#         for node in node_list_t:
#             t = ax.text((pos_t[node][1]-shift[1])//compress, (pos_t[node][0]-shift[0])//compress, str(relabel_t(node)), ha="center", va="center",
#                         size=size,
#                         bbox=bbox_props1)
#         for node in node_list_tp1:
#             if node in pos_tp1.keys():
#                 t = ax.text((pos_tp1[node][1]-shift[1])//compress, (pos_tp1[node][0]-shift[0])//compress, str(relabel_tp1(node)), ha="center", va="center",
#                             size=size,
#                             bbox=bbox_props2)
        if len(save)>=1:
            plt.savefig(save)
            plt.close(fig)
        else:
            plt.show()
        

class Node():
    def __init__(self,label,experiment):
        self.experiment = experiment
        self.label = label
    def __repr__(self):
        return (f'Node({self.label})')
    def __str__(self):
        return str(self.label)
    def neighbours(self,t):
        return([self.experiment.get_node(node) for node in self.experiment.nx_graph[t].neighbors(self.label)])
    def is_in(self,t):
        return(self.label in self.experiment.nx_graph[t].nodes)
    def degree(self,t):
        return(self.experiment.nx_graph[t].degree(self.label))
    def edges(self,t):
        return([self.experiment.get_edge(self,neighbour) for neighbour in self.neighbours(t)])
    def pos(self,t):
        return(self.experiment.positions[t][self.label])

class Edge():
    def __init__(self,begin,end,experiment):
        self.begin=begin
        self.end = end
        self.experiment = experiment
    def __repr__(self):
        return (f'Edge({self.begin},{self.end})')
    def __str__(self):
        return str((self.begin,self.end))
    def is_in(self,t):
        return((self.begin,self.end) in self.experiment.nx_graph[t].edges)
    def pixel_list(self,t):
        return(orient(self.experiment.nx_graph[t].get_edge_data(self.begin.label,self.end.label)['pixel_list'],self.begin.pos(t)))
    def orientation_whole(self,t):
        pixel_list = np.array(self.pixel_list(t))
        vector = pixel_list[-1]-pixel_list[0]
        unit_vector = vector/np.linalg.norm(vector)
        vertical_vector=np.array([-1,0])
        dot_product = np.dot(vertical_vector,unit_vector)
        if vertical_vector[1]*vector[0]-vertical_vector[0]*vector[1]>=0: #determinant
            angle = np.arccos(dot_product)/(2*np.pi)*360
        else:
            angle = -np.arccos(dot_product)/(2*np.pi)*360
        return(angle)
        

In [256]:
experiment = Experiment(13)

In [258]:
experiment.load(dates,local=False)

In [259]:
experiment_updated= Experiment(13)
experiment_updated.copy(experiment)

In [260]:
# fig=plt.figure(figsize=(9,9))
# ax = fig.add_subplot(111)
# np.random.seed(0)
# for t in ts:
#     color = list(np.random.choice(range(256), size=3))
experiment_updated.plot([0,1,2,3,4,5,6],compress=5)
# ax.imshow(0.8*experiment.compress_skeleton(1,5),cmap='plasma',interpolation='none',vmin=0,vmax=1)
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [247]:
experiment_updated

In [243]:
experiment_updated.pickle_save()

In [246]:
experiment_updated.pickle_load()

In [221]:
plt.close('all')

In [124]:
experiment.get_node(19).edges(0)[1].orientation_whole(0)

62.300527191945015

In [120]:
experiment.get_node(19).edges(0)

[Edge(19,25), Edge(19,16), Edge(19,20)]

In [44]:
list(experiment.nodes[].neighbours(0))

[2]

In [16]:
experiment.positions[0][0]

array([9248, 6724])

In [24]:
experiment.save()

wow


In [139]:
np.random.seed(1)
random_color()

[37, 235, 140]

In [133]:
import random
def random_color():
    return list(np.random.choice(range(256), size=3))

In [237]:
import sys

def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size